# 量子コンピューターで農地収穫量を最適化しよう

Yuri Kobayashi (Aug 22, 2023)

## はじめに <a id='Introduction'></a>

量子コンピューターは、古典コンピューターでは解けない問題を解決する可能性を秘めています。

実社会で有用な計算において量子コンピューターが古典コンピューターの計算速度を上回り、その優位性を示す領域を特定することは、量子コンピューターが実用化される時代に向けてとても重要なことです。このような優位性が期待される領域には、機械学習、計算化学、組み合わせ最適化問題などが挙げられます。今回は農地の収穫量を題材に、組み合わせ最適化問題を取り上げます。このハンズオンを通じ、量子コンピューターの優位性と今抱える問題について理解することができるでしょう。

### まずは、ハンズオンを実行する環境を準備しましょう。

1. Jupyter notebookのダウンロード<br/>
  ハンズオンで使用するJupyter notebookファイルをダウンロードします

2. [IBM Quantum](https://quantum-computing.ibm.com/)へのログイン

3. ステップ1でダウンロードしたファイルのアップロード

  3-1. [Dashbord](https://quantum-computing.ibm.com/)上の[Launch Lab](https://quantum-computing.ibm.com/lab)ボタンをクリックします
  
  3-2. Upload Filesボタン(上矢印)を押して、ステップ1でダウンロードしたファイルをアップロードします <br/>

4. アップロードしたファイル「20230822_optimization.ipynb」を開いてください

### 準備：必要なモジュールとライブラリのインポート
次に、必要なモジュールとライブラリをインポートしておきましょう。Jupyter notebookでは、セルにカーソルを置き、Shift+Enterを押すと、セル内のコードが実行されます。

それでは、まずはインストールされているQiskitのバージョンを調べてみましょう。バージョンを確認するには、qiskitライブラリをインポートしてから、以下のようなコマンドを実行します。

In [ ]:
import qiskit
qiskit.__qiskit_version__

In [ ]:
# 補助ライブラリのインポート
import numpy as np

# Qiskitのインポート
from qiskit import IBMQ, Aer
from qiskit.algorithms.minimum_eigensolvers import VQE, SamplingVQE, QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.providers.ibmq import least_busy
from qiskit.utils import QuantumInstance
from qiskit.tools import job_monitor

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo

from qiskit.circuit.library import TwoLocal
from qiskit.primitives import Sampler
from qiskit.quantum_info import Pauli, Statevector
from qiskit.result import QuasiDistribution
from qiskit.utils import algorithm_globals

以上で準備は完了です。

## 農地の収穫量問題 <a id="CropYieldProblem"></a>
農場の作物や経営を最適化して、リスクを減らしながら利益を上げたいというニーズはよくあります。アフリカや全世界が直面している大きな課題の1つは、すべての人に十分な食料をいかに生産するかということです。今回問題とするのは、利益ではなく、収穫した作物のトン数です。農地の広さが決められ、植えられる作物の種類が与えられたとき、どの作物をそれぞれ何ヘクタール植えたら、収穫量を最大にできるかが課題になります。

<table>
    <tr>
        <th>
            <img src="farm_template.svg" width="384px"/>
        </th>
    </tr>
    <tr>
        <th>
            あなたの経営する美しい3ha(ヘクタール)の農場
        </th>
    </tr>
</table>

<table>
    <tr>
        <th>
        <img src="crop_wheat.svg" width="256px"/>
        </th>
        <th>
            <img src="crop_soybeans.svg" width="256px"/>
        </th>
        <th>
            <img src="crop_maize.svg" width="256px"/>
        </th>
        <th>
            <img src="crop_pushpull.svg" width="256px"/>
        </th>
    </tr>
    <tr>
        <th>
            小麦
        </th>
        <th>
            大豆
        </th>
        <th>
            トウモロコシ
        </th>
        <th>
            プッシュプル
        </th>
<!--         <th>
            <p align="right" style="height:32px;padding-top:10px;">Wheat<img src="wheat.svg" width="32px" style="float:left;margin-top:-10px;margin-right:8px;"/></p>
        </th>
        <th>
            <p style="height:32px;padding-top:10px;">Soybeans<img src="soybeans.svg" width="32px" style="float:left;margin-top:-10px;margin-right:8px;"/></p>
        </th>
        <th>
            <p style="height:32px;padding-top:10px;">Maize<img src="maize.svg" width="32px" style="float:left;margin-top:-10px;margin-right:8px;"/></p>
        </th>
        <th>
            <p style="height:32px;padding-top:10px;">Push-Pull<img src="pushpull.svg" width="32px" style="float:left;margin-top:-10px;margin-right:8px;"/></p>
        </th> -->
    </tr>
</table>
農法には「単作」「間作」「プッシュプル農法」の3種類があります。単作とは、一つの作物だけを栽培する方法です。単作では、病気や害虫の影響を受けやすく、収穫量全体に影響を及ぼします。また、異なる作物を近くで栽培すると、両方の収穫量が増えたり、逆に収穫量が減ったりします。間作とは、収穫量を増やすために2つの異なる作物を選択することです。プッシュプル農法とは、害虫を寄せ付けないプッシュ作物と、害虫を引き寄せるプル作物をペアで栽培することです。これを大規模な農場に組み込むことで、収穫量を増やすことができます。ただ、プッシュプル作物は利用できなかったり食用にならないため、プッシュプル作物を全体の収穫量として使用できません。
<table>
    <tr>
        <th>
        <img src="farm_mono.svg" width="256px"/>
        </th>
        <th>
            <img src="farm_intercrop.svg" width="256px"/>
        </th>
        <th>
            <img src="farm_intercrop_pushpull.svg" width="256px"/>
        </th>
    </tr>
    <tr>
        <th>
            単作
        </th>
        <th>
            間作
        </th>
        <th>
            プッシュプル農法
        </th>
    </tr>
</table>

このような農地の収穫量問題は、変数の特定の組み合わせで解が得られるという点で、組合せ最適化問題です。ここで紹介する問題は古典的に解くことができるほど小さいものですが、より大きな問題になると、最適化すべき組み合わせの数が多くなるため、古典的なコンピュータでは扱いにくくなります。

### 目的：収穫量の最大化
あなたは$3~ha$の農場のオーナーです。植えられる作物は、小麦、大豆、トウモロコシ、プッシュプルの4種類です。プッシュプルは、収穫しても売ることはできませんが、他の作物の収穫量を増やすことができます。それぞれの作物は$0~ha$または$1~ha$作付けすることができます。この農場の収穫量(トン)を以下のような2次関数として定義します。この2次関数の変数は、作付けする作物の作付け面積（ヘクタール数）です。解きたい問題は、どの作物の組合せを選べば、最大の収穫量が得られるか(2次関数を最大化できるか)ということです。

#### 作物の種類
<img src="qubo_problem_graphical_variables.svg" width="534px"/> <br>
$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$$~$小麦　　　　　　　大豆　　　　　　トウモロコシ　　　　　プッシュプル

#### 問題の定式化
<img src="qubo_problem_graphical.svg" width="400px"/>


$$
\begin{align}
    \text{maximize} \quad & 2(\operatorname{Wheat}) + \operatorname{Soybeans} + 4(\operatorname{Maize}) \\
    & + 2.4(\operatorname{Wheat}\times\operatorname{Soybeans}) \\ & + 4(\operatorname{Wheat}\times\operatorname{Maize})\\
    &+ 4(\operatorname{Wheat}\times\operatorname{PushPull}) \\ & + 2(\operatorname{Soybeans}\times\operatorname{Maize}) \\
                          & + (\operatorname{Soybeans}\times\operatorname{PushPull}) \\ & + 5(\operatorname{Maize}\times\operatorname{PushPull})
\end{align}
$$

$$
\begin{align}
\text{subject to} \quad & \operatorname{Wheat} + \operatorname{Soybeans} + \operatorname{Maize} + \operatorname{PushPull} \leq{} 3\\
& 0\leq{}\operatorname{Wheat}\leq{}1\\
& 0\leq{}\operatorname{Soybeans}\leq{}1\\
& 0\leq{}\operatorname{Maize}\leq{}1\\
& 0\leq{}\operatorname{PushPull}\leq{}1
\end{align}
$$

### 演習：量子コンピューターが計算できる形２次計画問題を作成してみましょう

上記で定義した問題を表す２次計画問題（`QuadraticProgram`）を作成するための準備として、 `cropyield_quadratic_program` 関数を定義します。つづいて、上記の目的関数にある各変数の係数をみながら、以下の「**#目的関数の定義**」と「**#制約の追加**」の各コメント下の式にそれぞれ係数を代入してみましょう。

変数名は何でもよいですが、ここでは `麦`、 `大豆`、 `トウモロコシ`、 および `プッシュプル`を使っています。

In [ ]:
#穴埋め問題
# 先に、２次計画問題 (Quadratic Program)の作成準備をしておきます。
def cropyield_quadratic_program():
    cy = QuadraticProgram(name="Crop Yield") #「穀物収穫量」を意味する'crop yield'の頭文字を取って'cy' と名付けます
  
    # 変数の追加
    cy.binary_var(name="麦") #Wheatの変数
    cy.binary_var(name="大豆") #Soybeansの変数
    cy.binary_var(name="トウモロコシ") #Maizeの変数
    cy.binary_var(name="プッシュプル") #Pushpullの変数
    
# 問題の定式化(穴埋め問題)
    ##############################
    
    # 定式化のパートはここから。単作と間作の係数をそれぞれ当てはめてください。
    
    # 目的関数の定義
    cy.maximize(
        linear={"麦": , "大豆": , "トウモロコシ": },  #1次式:単作の係数をあてはめます
        quadratic={("麦", "大豆"):  , ("麦", "トウモロコシ"):  , ("麦", "プッシュプル"):   ,("大豆", "トウモロコシ"):  , ("大豆", "プッシュプル"): , ("トウモロコシ", "プッシュプル"): },
    ) #2次式:間作の係数をあてはめます
    
    # 制約の追加
    cy.linear_constraint(linear={"麦": , "大豆": ,"トウモロコシ": , "プッシュプル":1 }, sense="<=", rhs=3)
    
    
    ##############################
    return cy

# 問題をLP文字列で出力
cy = cropyield_quadratic_program()
# print(cropyield.export_as_lp_string())
print(cy.prettyprint())

## 量子コンピューターで解くための準備 <br/> QUBO形式への変換 <a id="QUBO"></a>


最適化問題を特定の量子アルゴリズムで解くためには、通常、適性なフォーマットに変換して適用可能な状態にする必要があります。今回のような2次計画問題を量子コンピューター解く際には、Quadratic Unconstrained Binary Optimization (QUBO) 形式という特殊な形式に変換します。

興味深いことに、この農地収穫量最適化問題は、ハミルトニアンの基底状態(一番低いエネルギー状態）を求める問題として解くことができます。ハミルトニアンとは、シミュレーションしたい物理系の全エネルギーを表すエネルギー関数と考えることができます。この物理系は、さらに [**イジングモデル**](https://en.wikipedia.org/wiki/Ising_model) と呼ばれる数学モデルで表現することができます。この数学モデルは、電子のスピンの上向き・下向きを、バイナリ変数の0、1に読み替えることで、QUBO問題をイジングモデルにマップするのです。
    
今回適用するVQEやQAOAのような変分量子アルゴリズムは[**二次非制約二次最適化(QUBO)**](https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization)という形式に問題を変換する必要があり、 Qiskitはこの変換機能を提供します。

## 演習: 農地収穫量問題のQUBOへの変換
私たちの農地収穫量問題をQUBOに変換しましょう。
最適化問題を量子コンピューターが解けるよう、QUBO形式に変換してくれるクラス[QuadraticProgramToQubo](https://qiskit.org/ecosystem/optimization/stubs/qiskit_optimization.converters.QuadraticProgramToQubo.html)を呼び出し、先ほど作成した２次計画問題の`cy`を引数に変換(convert)します。

In [ ]:
# 農地収穫量問題をQUBO形式に変換する

cyQUBO=QuadraticProgramToQubo().convert(cy)

# print(cyQUBO.prettyprint())
print(cyQUBO.export_as_lp_string())

## 古典的な解法 <a id="ClassicalSolution"></a>

以上で、農地収穫量の最適化問題を量子コンピューターで解ける形に定義することができました。この問題はさほど大きくないので、古典的に厳密解を求めることも可能です。実際に量子コンピューターで解く前に、古典的に解を求めておきましょう。

古典的な解は、NumpyとQiskitを使って簡単に得ることができます。QUBO問題を解くことは、基礎となる行列表現の最小固有値を見つけることに他なりません。そして幸いなことに、この行列表現がどのようなものかを知る必要はなく、定義したQUBO問題を、`NumPyMinimumEigensolver`と`MinimumEigenOptimizer`に渡すだけでよいのです。

古典解を求める関数は以下のように定義できます。まずソルバーとして`NumPyMinimumEigensolver`インスタンスを持つ、`MinimumEigenOptimizer` オプティマイザーを作成します。与えられた2次計画問題を引数として`solve` メソッドを呼ぶと、オプティマイザーは，問題をパラメーター化された表現に変換してソルバーに渡し、ソルバーはパラメーターを最適化することで、行列表現の最小固有値を求め、最終的に元の問題の解を得ることができます。

しかし、通常は自分でやる必要はありません。Qiskitの最適化がやってくれます。アルゴリズムの一例としてMinimumEigenOptimizerを紹介します。QuadraticProgramを入力とし、最適解（必ずしも最適ではない）を計算します。また、量子回路の結果（通常はビット列）を変数に解釈する。以下の例ではQAOAを使って問題を解きます。

In [ ]:
def get_classical_solution_for(quadprog: QuadraticProgram):
    # Create solver
    solver = NumPyMinimumEigensolver()

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Return result from optimizer
    return optimizer.solve(quadprog)

私たちの農地収量問題を解いてみましょう。

In [ ]:
# 古典解を求める
classical_result = get_classical_solution_for(cy)

# Format and print result
print("古典の解法で求める参考値\n")
print(f"最大収穫量は {classical_result.fval} トン")
print(f"作付けすべき作物は: ")

_crops = [v.name for v in cy.variables]
for cropIndex, cropHectares in enumerate(classical_result.x):
    print(f"\t{_crops[cropIndex]}を{cropHectares}ha")

##  量子コンピューター上で回路を実行してみましょう

ここでは量子プログラム開発をよりシンプルに、そしてプログラムをより高速に実行するためのサービスであるQiskit Runtimeという環境を使って実行します。そのためにまずは必要なライブラリをインポートします。

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Options

In [ ]:
seed = 1234
algorithm_globals.random_seed = seed
service = QiskitRuntimeService(instance="ibm-q-education/ibm-3/kawasaki-camp")  # set an appropriate instance
backend = "ibmq_guadalupe"
options = Options(optimization_level=3, resilience_level=1)
options.execution.shots = 1000
reps = 2
maxiter = 100
with Session(service=service, backend=backend) as session:
    sampler = Sampler(session=session, options=options)
    qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=maxiter), reps=reps)
    min_eigen_optimizer = MinimumEigenOptimizer(qaoa)
    result = min_eigen_optimizer.solve(cyQUBO)
print(result)

量子コンピューターで実行した結果はいかがでしょうか？古典解と同じ結果がでたでしょうか？無事に計算が実行され完了した場合は下記のような計算結果が得られたはずです。


'麦=1.0, 大豆=0.0, トウモロコシ=1.0, プッシュプル=1.0　<<省略>> status=SUCCESS'

## まとめ <a id="Summary"></a>

いかがでしたか？本日はハンズオンを通して以下を説明しました：

- 最適化問題を解くための流れ
- 定式化からの2次計画問題作成方法（係数の入力）
- 2次計画問題のQUBO形式への変換
- 量子アルゴリズム(を適用した解法

量子コンピューターが最適化問題の解法として優越性が期待されているものの、実際の問題を解くためには、量子ビット数を削減できるようなモデルに落とし込むことが重要です。

この問題が取り上げられたQuantum Challenge 2021 Africaの問題は[こちら](https://github.com/qiskit-community/ibm-quantum-challenge-africa-2021)から取得可能です。このハンズオンでご紹介した問題のほか、金融や創薬の問題があります。また先日終了した[Quantum Challenge 2021 Fall](https://github.com/qiskit-community/ibm-quantum-challenge-fall-2021)にも興味深い問題がたくさんあります。こちらは日本語訳もありますので、より取り組みやすいと思います。来年もChallengeがあると予想されます。規定数の問題を解くとバッジが取得できますので、みなさまもぜひ挑戦してみてください！

また、Qiskitをもっと知りたい方には、Qiskit Documentationの[Tutorial](https://qiskit.org/documentation/tutorials.html)を、量子コンピューターについてもっと勉強したい方には、[Qiskit Textbook](https://qiskit.org/textbook/preface.html)をお勧めします。TutorialはIBM Quantumの[Quantum Lab](https://quantum-computing.ibm.com/lab)からも参照＆実行できます。

## 参考文献 <a id="Reference"></a>
[1] A. A. Nel, ‘Crop rotation in the summer rainfall area of South Africa’, South African Journal of Plant and Soil, vol. 22, no. 4, pp. 274–278, Jan. 2005, doi: 10.1080/02571862.2005.10634721.

[2] H. Ritchie and M. Roser, ‘Crop yields’, Our World in Data, 2013, [Online]. Available: https://ourworldindata.org/crop-yields.

[3] G. Brion, ‘Controlling Pests with Plants: The power of intercropping’, UVM Food Feed, Jan. 09, 2014. https://learn.uvm.edu/foodsystemsblog/2014/01/09/controlling-pests-with-plants-the-power-of-intercropping/ (accessed Feb. 15, 2021).

[4] N. O. Ogot, J. O. Pittchar, C. A. O. Midega, and Z. R. Khan, ‘Attributes of push-pull technology in enhancing food and nutrition security’, African Journal of Agriculture and Food Security, vol. 6, pp. 229–242, Mar. 2018.

In [ ]:
import qiskit.tools.jupyter

#%qiskit_version_table
%qiskit_copyright